In [20]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import classification_report
import timm

DATA_ROOT = r"E:\semester 7\DL\tubes_2\data"
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
VAL_DIR   = os.path.join(DATA_ROOT, "val")

BATCH_SIZE = 8
NUM_EPOCHS = 25
LR = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [16]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
val_dataset   = datasets.ImageFolder(VAL_DIR, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

num_classes = len(train_dataset.classes)
num_classes


69

In [17]:
model = timm.create_model(
    "swin_tiny_patch4_window7_224",
    pretrained=True,
    num_classes=num_classes
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR)


In [21]:
best_val_acc = 0.0

for epoch in range(NUM_EPOCHS):
    # TRAINING
    model.train()
    correct = 0
    total = 0
    running_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # VALIDATION
    model.eval()
    correct = 0
    total = 0
    running_loss = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    val_acc = correct / total

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "models/swin_best.pth")

    print(f"[{epoch+1}/{NUM_EPOCHS}] Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f}")

print("Best Val Acc:", best_val_acc)


[1/25] Train Acc: 0.995 | Val Acc: 0.600
[2/25] Train Acc: 1.000 | Val Acc: 0.600
[3/25] Train Acc: 0.995 | Val Acc: 0.614
[4/25] Train Acc: 1.000 | Val Acc: 0.586
[5/25] Train Acc: 1.000 | Val Acc: 0.600
[6/25] Train Acc: 0.995 | Val Acc: 0.614
[7/25] Train Acc: 1.000 | Val Acc: 0.571
[8/25] Train Acc: 1.000 | Val Acc: 0.586
[9/25] Train Acc: 1.000 | Val Acc: 0.586
[10/25] Train Acc: 1.000 | Val Acc: 0.600
[11/25] Train Acc: 1.000 | Val Acc: 0.600
[12/25] Train Acc: 1.000 | Val Acc: 0.600
[13/25] Train Acc: 0.995 | Val Acc: 0.600
[14/25] Train Acc: 1.000 | Val Acc: 0.600
[15/25] Train Acc: 1.000 | Val Acc: 0.600
[16/25] Train Acc: 0.995 | Val Acc: 0.600
[17/25] Train Acc: 0.995 | Val Acc: 0.600
[18/25] Train Acc: 0.995 | Val Acc: 0.571
[19/25] Train Acc: 1.000 | Val Acc: 0.586
[20/25] Train Acc: 1.000 | Val Acc: 0.586
[21/25] Train Acc: 1.000 | Val Acc: 0.614
[22/25] Train Acc: 0.995 | Val Acc: 0.586
[23/25] Train Acc: 1.000 | Val Acc: 0.614
[24/25] Train Acc: 0.995 | Val Acc: 0.600
[

In [22]:
print(classification_report(
    all_labels,
    all_preds,
    target_names=train_dataset.classes
))


                                 precision    recall  f1-score   support

           Abraham Ganda Napitu       0.00      0.00      0.00         1
       Abu Bakar Siddiq Siregar       1.00      1.00      1.00         1
             Ahmad Faqih Hasani       0.00      0.00      0.00         1
                   Aldi Sanjaya       0.00      0.00      0.00         1
                        Alfajar       1.00      1.00      1.00         1
            Alief Fathur Rahman       0.00      0.00      0.00         1
 Arkan Hariz Chandrawinata Liem       0.50      1.00      0.67         1
               Bayu Ega Ferdana       0.50      1.00      0.67         1
          Bayu Prameswara Haris       0.00      0.00      0.00         1
           Bezalel Samuel Manik       1.00      1.00      1.00         1
           Bintang Fikri Fauzan       0.00      0.00      0.00         1
              Boy Sandro Sigiro       1.00      1.00      1.00         1
             Desty Ananta Purba       0.50      1.

e:\semester 7\DL\tubes_2\venv\lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
e:\semester 7\DL\tubes_2\venv\lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
e:\semester 7\DL\tubes_2\venv\lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
e:\semester 7\DL\tubes_2\venv\lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The numbe

In [ ]:
import json

label_map = train_dataset.class_to_idx  # dari ImageFolder
inv_label_map = {v: k for k, v in label_map.items()}

with open("models/label_map_swin.json", "w") as f:
    json.dump(inv_label_map, f)

print("Label map saved!")


Label map saved!
